In [1]:
import pyspark
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

22/05/23 11:21:41 WARN Utils: Your hostname, CHRISs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.8 instead (on interface en0)
22/05/23 11:21:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/23 11:21:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/23 11:21:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


1. Read the case, department, and source data into their own spark dataframes.



In [6]:
case_df = spark.read.csv('case.csv', header=True, inferSchema=True)
department_df = spark.read.csv('dept.csv', header=True, inferSchema=True)
source_df = spark.read.csv('source.csv', header=True, inferSchema=True)

2. Let's see how writing to the local disk works in spark:



    a. Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json


In [7]:
# store source_df as csv and json
source_df.write.csv('sources_csv.csv')
source_df.write.json('source_json.json')

    b. Inspect your folder structure. What do you notice?


new folders are created with the json and csv files contained within them.


3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.


In [12]:
case_df.show(2, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [23]:
# convert column name
case_df = case_df.withColumnRenamed("SLA_due_date", "case_due_date")
#convert to boolean
case_df = case_df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)
#convert to string
case_df = case_df.withColumn('council_district', format_string('%04d', col('council_district')))

#convert to date timestamp
case_df = (
    case_df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)
# Create a `case_lifetime` feature
case_df = (
    case_df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)



***



1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?


In [26]:
case_df.createOrReplaceTempView('df')

spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 1
''').show()

+-------------+
|days_past_due|
+-------------+
|         1952|
+-------------+



2. How many Stray Animal cases are there?


In [28]:
case_df.filter(case_df.service_request_type == 'Stray Animal').count()

26760

3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?



In [30]:
(
    case_df.filter(case_df.dept_division == 'Field Operations')
    .filter(case_df.service_request_type != 'Officer Standby')
    .count()
)

113902

4. Convert the council_district column to a string column.



In [31]:
# done above

5. Extract the year from the case_closed_date column.


In [32]:
case_df.select('case_closed_date', year('case_closed_date')).show(5)


+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



6. Convert num_days_late from days to hours in new columns num_hours_late.



In [33]:
(
    case_df.withColumn('num_hours_late', case_df.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show(5)
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



7. Join the case data with the source and department data.



In [35]:
df = case_df.join(department_df, 'dept_division', 'left').join(source_df, 'source_id', 'left')
df.show(1, vertical=True)


-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 case_due_date          | 2020-09-26 00:42:00  
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | true                 
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 0005                 
 case_lifetime          | 0                    
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS             
only showing top 1 row



8. Are there any cases that do not have a request source?



In [36]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



9. What are the top 10 service request types in terms of number of requests?



In [37]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



10. What are the top 10 service request types in terms of average days late?


In [40]:
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(round(mean('num_days_late'),1).alias('n_days_late'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+-----------+
|service_request_type                  |n_days_late|
+--------------------------------------+-----------+
|Zoning: Recycle Yard                  |210.9      |
|Zoning: Junk Yards                    |200.2      |
|Structure/Housing Maintenance         |190.2      |
|Donation Container Enforcement        |171.1      |
|Storage of Used Mattress              |164.0      |
|Labeling for Used Mattress            |162.4      |
|Record Keeping of Used Mattresses     |154.0      |
|Signage Requied for Sale of Used Mattr|151.6      |
|Traffic Signal Graffiti               |137.6      |
|License Requied Used Mattress Sales   |128.8      |
+--------------------------------------+-----------+
only showing top 10 rows



11. Does number of days late depend on department?


In [43]:
(
    df.filter("case_closed")
    .filter("case_late")
    .groupby("standardized_dept_name")
    .agg(avg("num_days_late").alias("days_late"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(col('standardized_dept_name'), col('days_late'))
    .show(40, truncate=False)
)

+------------------------+---------+
|standardized_dept_name  |days_late|
+------------------------+---------+
|Animal Care Services    |23.5     |
|Customer Service        |81.4     |
|DSD/Code Enforcement    |45.3     |
|Metro Health            |6.6      |
|Parks & Recreation      |20.2     |
|Solid Waste             |6.6      |
|Trans & Cap Improvements|7.1      |
+------------------------+---------+



12. How do number of days late depend on department and request type?

In [47]:
(
    df.filter("case_closed")
    .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(col('days_late').desc(), col('standardized_dept_name'))``
    .show(40, truncate=False)
)

+----------------------+--------------------------------------------+---------+
|standardized_dept_name|service_request_type                        |days_late|
+----------------------+--------------------------------------------+---------+
|DSD/Code Enforcement  |Zoning: Recycle Yard                        |273.6    |
|DSD/Code Enforcement  |Zoning: Junk Yards                          |251.9    |
|DSD/Code Enforcement  |Donation Container Enforcement              |201.7    |
|DSD/Code Enforcement  |Structure/Housing Maintenance               |182.4    |
|DSD/Code Enforcement  |Graffiti: Private Property (Corridors)      |175.1    |
|DSD/Code Enforcement  |Storage of Used Mattress                    |164.0    |
|DSD/Code Enforcement  |Labeling for Used Mattress                  |162.4    |
|DSD/Code Enforcement  |Record Keeping of Used Mattresses           |154.0    |
|DSD/Code Enforcement  |Signage Requied for Sale of Used Mattr      |151.6    |
|DSD/Code Enforcement  |License Requied 